# Plateforme Agnostique de Traitement et d'Analyse des Textes
### Carnet d'expérimentation
---

## Sujet : Analyse Recueil
- Environnement
- Expérience
- Résultats

---

# Observations et environnement
---

In [1]:
random_state = 42

In [2]:
cd ../..

/Volumes/Geek/Work/Patat


In [3]:
import importlib
import pandas as pd
import seaborn as sns

In [4]:
from tqdm.notebook import tqdm
tqdm.pandas()

## Recueil Brut

In [5]:
import patat.util.gsheet

gs = patat.util.gsheet.GoogleSheet()

sheet_id = '1RFxwIBYqa_eYcx_LZpTdhi47QbLcesz9NByMfGzdJ8s'
range_name = 'articles labellisés'

df_brut = gs.read_df(sheet_id,range_name)

df_brut

,#site,# article,Site,url,Auteur/trice,Date,Titre,Explication (pour les infox seulement),infox,signé,...,inversion de la preuve,invérifiable,titre décalé,dénigrement,cherry picking,degré infox,observateur,thème (abandonné),type infox (abandonné),Date de collection (abandonné)
0,1,1,Alnas,https://www.alnas.fr/actualite/communaute/lara...,,31/10/2022,L’Arabie saoudite organise des célébrations d'...,,non,non,...,non,non,non,non,non,,FB,None,None,None
1,1,2,Alnas,https://www.alnas.fr/actualite/en-vrac/quel-ar...,,19/10/2022,« Quel art ! » : un étudiant en droit invente ...,,non,non,...,non,non,non,non,non,,FB,None,None,None
2,1,3,Alnas,https://www.alnas.fr/actualite/politique/laust...,,18/10/2022,L’Australie ne reconnaît plus Jérusalem comme ...,,non,non,...,non,non,non,non,non,,FB,None,None,None
3,1,4,Alnas,https://www.alnas.fr/actualite/communaute/nant...,,17/10/2022,Nantes : une mère musulmane meurt poignardée e...,,non,non,...,non,non,non,non,non,,FB,None,None,None
4,1,5,Alnas,https://www.alnas.fr/actualite/en-vrac/la-fran...,,10/11/2022,La France condamnée pour avoir placé un enfant...,le titre est décalé (donne une impression faus...,non,non,...,non,non,oui,non,non,,FB,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,44,618,Wikistrike,https://www.wikistrike.com/2022/10/la-pologne-...,,31/10/2022,La Pologne s’apprête à s’emparer du territoire...,relai de ragots invérifiés,oui,non,...,non,oui,oui,non,non,2,FB,None,None,None
619,44,619,Wikistrike,https://www.wikistrike.com/2022/11/exclusivite...,,19/11/2022,Exclusivité WikiStrike: un physicien en colère...,délire total d'un physicien anonyme (ahurissan...,oui,non,...,non,non,non,non,non,1,FB,None,None,None
620,44,620,Wikistrike,https://www.wikistrike.com/2022/12/zelensky-pa...,,23/12/2023,Zelensky parti demander à papa Biden une aide ...,l'article est vide (même si le titre accuse Ze...,oui,non,...,non,non,non,non,non,1,FB,None,None,None
621,44,621,Wikistrike,https://www.wikistrike.com/2023/01/le-chef-du-...,,03/01/2023,Le chef du renseignement iranien assure que la...,long compte rendu d'un interview du chef du re...,non,non,...,non,non,non,non,non,,FB,None,None,None


## Normalisation Recueil Brut

In [6]:
from patat.imp.labels import label_translator
from patat.imp.labels import short_label_names

df_recueil = df_brut.rename(columns=label_translator)
df_label = df_recueil[['url','explication'] + short_label_names].copy()

import numpy as np
def normalize_label(value):
    if type(value) != str:
        return np.NaN
    elif value == 'oui':
        return 1.0
    elif value =='non' :
        return 0.0
    else:
        return np.NaN

for label in short_label_names:
    df_label[label]=df_label[label].apply(normalize_label)

df_label

,url,explication,infox,signe,entites_nommees,entites_coherentes,ouverture_esprit,qualite_ecriture,faits,opinions,...,trop_mots,trop_chiffres,sophisme,inversion_preuve,inverifiable,propos_raportes,sources_citees,titre_decale,denigrement,cherry_picking
0,https://www.alnas.fr/actualite/communaute/lara...,,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,https://www.alnas.fr/actualite/en-vrac/quel-ar...,,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,https://www.alnas.fr/actualite/politique/laust...,,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,https://www.alnas.fr/actualite/communaute/nant...,,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,https://www.alnas.fr/actualite/en-vrac/la-fran...,le titre est décalé (donne une impression faus...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,https://www.wikistrike.com/2022/10/la-pologne-...,relai de ragots invérifiés,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0
619,https://www.wikistrike.com/2022/11/exclusivite...,délire total d'un physicien anonyme (ahurissan...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
620,https://www.wikistrike.com/2022/12/zelensky-pa...,l'article est vide (même si le titre accuse Ze...,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
621,https://www.wikistrike.com/2023/01/le-chef-du-...,long compte rendu d'un interview du chef du re...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


## Importation des contenus et meta données

In [19]:
filename = 'data/tmp/230506-Recueil.csv'

In [20]:
df_articles = pd.read_csv(filename)

In [21]:
df_articles

,url,site,title,article,author,date,tags,paragraphs,date_iso,abstract,c_count,p_count
0,https://www.alnas.fr/actualite/communaute/lara...,www.alnas.fr,L’Arabie saoudite organise des célébrations d'...,L’Arabie saoudite a organisé un événement pour...,Par Youcef -,31/10/2022,['ActualitésCommunauté'],['L’Arabie saoudite a organisé un événement po...,2022-10-31T00:00:00,NaN,2277,8
1,https://www.alnas.fr/actualite/en-vrac/quel-ar...,www.alnas.fr,« Quel art ! » : un étudiant en droit invente ...,Une professeure de droit a partagé sur Twitter...,Par Oussama -,19/10/2022,['ActualitésEn vrac'],['Une professeure de droit a partagé sur Twitt...,2022-10-19T00:00:00,NaN,1683,8
2,https://www.alnas.fr/actualite/politique/laust...,www.alnas.fr,L’Australie ne reconnaît plus Jérusalem comme ...,Le gouvernement israélien a qualifié la positi...,Par Oussama -,18/10/2022,['ActualitésPolitique'],['Le gouvernement israélien a qualifié la posi...,2022-10-18T00:00:00,NaN,2678,15
3,https://www.alnas.fr/actualite/communaute/nant...,www.alnas.fr,Nantes : une mère musulmane meurt poignardée e...,NANTES – Une enquête pour « homicide volontair...,Par Sarah -,17/10/2022,['ActualitésCommunauté'],['NANTES – Une enquête pour «\xa0homicide volo...,2022-10-17T00:00:00,NaN,1792,5
4,https://www.alnas.fr/actualite/en-vrac/la-fran...,www.alnas.fr,La France condamnée pour avoir placé un enfant...,La Cour européenne des droits de l’Homme (CEDH...,Par Sarah -,10/11/2022,['ActualitésEn vrac'],['La Cour européenne des droits de l’Homme (CE...,2022-11-10T00:00:00,NaN,1482,5
...,...,...,...,...,...,...,...,...,...,...,...,...
609,https://www.wikistrike.com/2022/10/selon-un-an...,www.wikistrike.com,"Selon un ancien conseiller du Pentagone, les É...","Selon un ancien conseiller du Pentagone, les c...",NaN,Publié par wikistrike.com\n ...,[],"['Selon un ancien conseiller du Pentagone, les...",2022-10-04T00:00:00,NaN,3976,14
610,https://www.wikistrike.com/2022/10/la-pologne-...,www.wikistrike.com,La Pologne s’apprête à s’emparer du territoire...,La Pologne se prépare à s’emparer du territoir...,NaN,Publié par wikistrike.com\n ...,[],['La Pologne se prépare à s’emparer du territo...,2022-10-31T00:00:00,NaN,2284,8
611,https://www.wikistrike.com/2022/11/exclusivite...,www.wikistrike.com,Exclusivité WikiStrike: un physicien en colère...,"Crise énergétique battant son plein, WikiStri...",NaN,Publié par wikistrike.com\n ...,[],"[""Crise énergétique battant son plein,\xa0\xa0...",2022-11-19T00:00:00,NaN,27223,27
612,https://www.wikistrike.com/2022/12/zelensky-pa...,www.wikistrike.com,Zelensky parti demander à papa Biden une aide ...,"Alors que des milliers d’Ukrainiens meurent, Z...",NaN,Publié par wikistrike.com\n ...,[],"['Alors que des milliers d’Ukrainiens meurent,...",2022-12-23T00:00:00,NaN,557,5


# Experience
---

## Articles et Titres vides

In [22]:
df_recueil = pd.merge(df_articles,df_label,how='outer')

In [23]:
df_recueil

,url,site,title,article,author,date,tags,paragraphs,date_iso,abstract,...,trop_mots,trop_chiffres,sophisme,inversion_preuve,inverifiable,propos_raportes,sources_citees,titre_decale,denigrement,cherry_picking
0,https://www.alnas.fr/actualite/communaute/lara...,www.alnas.fr,L’Arabie saoudite organise des célébrations d'...,L’Arabie saoudite a organisé un événement pour...,Par Youcef -,31/10/2022,['ActualitésCommunauté'],['L’Arabie saoudite a organisé un événement po...,2022-10-31T00:00:00,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,https://www.alnas.fr/actualite/en-vrac/quel-ar...,www.alnas.fr,« Quel art ! » : un étudiant en droit invente ...,Une professeure de droit a partagé sur Twitter...,Par Oussama -,19/10/2022,['ActualitésEn vrac'],['Une professeure de droit a partagé sur Twitt...,2022-10-19T00:00:00,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,https://www.alnas.fr/actualite/politique/laust...,www.alnas.fr,L’Australie ne reconnaît plus Jérusalem comme ...,Le gouvernement israélien a qualifié la positi...,Par Oussama -,18/10/2022,['ActualitésPolitique'],['Le gouvernement israélien a qualifié la posi...,2022-10-18T00:00:00,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,https://www.alnas.fr/actualite/communaute/nant...,www.alnas.fr,Nantes : une mère musulmane meurt poignardée e...,NANTES – Une enquête pour « homicide volontair...,Par Sarah -,17/10/2022,['ActualitésCommunauté'],['NANTES – Une enquête pour «\xa0homicide volo...,2022-10-17T00:00:00,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,https://www.alnas.fr/actualite/en-vrac/la-fran...,www.alnas.fr,La France condamnée pour avoir placé un enfant...,La Cour européenne des droits de l’Homme (CEDH...,Par Sarah -,10/11/2022,['ActualitésEn vrac'],['La Cour européenne des droits de l’Homme (CE...,2022-11-10T00:00:00,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620,https://planetes360.fr/la-france-vit-tous-les-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
621,https://francais.rt.com/france/103548-reforme-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
622,https://francais.rt.com/france/103613-reforme-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
623,https://francais.rt.com/economie/103615-export...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
for url in df_recueil[df_recueil['article'].isna()]['url']:
    print(url)

https://www.lumiere-dans-la-matrice.org/wp/la-science-sest-trompee-lhomme-ne-descend-pas-du-singe/
https://planetes360.fr/scoop-lancien-vice-president-de-pfizer-declare-que-les-failles-des-vaccins-covid-sont-intentionnelles/
https://planetes360.fr/exclu-360-le-professeur-perr0nne-balance-tout-c0vid-masque-vaxxin-bill-gates-la-verite-va-eclater/
https://planetes360.fr/urgent-votre-argent-les-interesse-un-reset-qui-pourrait-bien-ruiner-des-millions-de-francais/
https://planetes360.fr/la-france-vit-tous-les-symptomes-dun-effondrement-depuis-plusieurs-annees-maintenant-et-malheureusement-cela-risque-de-sempirer-et-la-chute-risque-de-saccelerer/
https://francais.rt.com/france/103548-reforme-retraites-inflation-crise-renseignement-interieur-predit-annee-difficile
https://francais.rt.com/france/103613-reforme-retraites-93-actifs-contre-report-age-legal-selon-institut-montaigne
https://francais.rt.com/economie/103615-exportations-chinoises-devissent-decembre-mais-echanges-avec-russie-restent-d

In [25]:
df_recueil = pd.merge(df_articles,df_label,how='inner')

In [26]:
for url in df_recueil[df_recueil['title'].isna()]['url']:
    print(url)

## Comptage Infox

In [27]:
df_recueil.pivot_table(index='site',columns='infox',values='article',aggfunc='count').fillna(0).astype(int).sort_values(1.0)





infox,0.0,1.0
site,,
www.alnas.fr,7,0
extime.fr,2,0
fr.novopress.info,6,0
www.cnews.fr,4,0
www.fdesouche.com,2,0
lumieresurgaia.com,4,0
qactus.fr,1,1
ripostelaique.com,7,1
www.bvoltaire.fr,5,1


In [28]:
for url in df_recueil[df_recueil['p_count']==0]['url']:
    print(url)

# Mesure et observation des résultats
---

# Sauvegarde
---

# Conclusions
---

- ...

# Bricolages
---

In [33]:
df_recueil['abstract'].fill

587